# Cargamos los datasets con los que vamos a trabajar

In [16]:
import pandas as pd
# Especifica la ruta del archivo CSV que deseas leer
file_path = 'Data/Datos_csv/games.csv'

# Lee el archivo CSV y crea un DataFrame
games = pd.read_csv(file_path, parse_dates=['release_date'])

# Ahora puedo trabajar con el DataFrame 'games'
print(games)

                 publisher                                             genres  \
0                Kotoshiro  ['Action', 'Casual', 'Indie', 'Simulation', 'S...   
1         Making Fun, Inc.       ['Free to Play', 'Indie', 'RPG', 'Strategy']   
2             Poolians.com  ['Casual', 'Free to Play', 'Indie', 'Simulatio...   
3                     彼岸领域                  ['Action', 'Adventure', 'Casual']   
4      Trickjump Games Ltd              ['Action', 'Adventure', 'Simulation']   
...                    ...                                                ...   
22491      Bidoniera Games         ['Action', 'Adventure', 'Casual', 'Indie']   
22492      Ghost_RUS Games      ['Casual', 'Indie', 'Simulation', 'Strategy']   
22493               Sacada                    ['Casual', 'Indie', 'Strategy']   
22494         Laush Studio                  ['Indie', 'Racing', 'Simulation']   
22495             SIXNAILS                                ['Casual', 'Indie']   

                       app_

In [17]:
# Especifica la ruta del archivo CSV que deseas leer
file_path = 'Data/Datos_csv/reviews_nlp.csv'

# Lee el archivo CSV y crea un DataFrame
reviews = pd.read_csv(file_path, parse_dates=['posted'])

# Ahora puedes trabajar con el DataFrame 'reviews'
print(reviews)

                 user_id     posted  item_id  recommend  \
0      76561197970982479 2011-11-05     1250       True   
1      76561197970982479 2011-07-15    22200       True   
2      76561197970982479 2011-04-21    43110       True   
3                js41637 2014-06-24   251610       True   
4                js41637 2013-09-08   227300       True   
...                  ...        ...      ...        ...   
49181            wayfeng 2015-10-14      730       True   
49182  76561198251004808 2015-10-10   253980       True   
49183        72947282842 2015-10-31      730       True   
49184          ApxLGhost 2015-12-14      730       True   
49185  76561198267374962 2015-12-29   369200       True   

                                                  review  sentiment_score  \
0      Simple yet with great replayability. In my opi...           0.8481   
1                   It's unique and worth a playthrough.           0.2263   
2      Great atmosphere. The gunplay can be a bit chu...    

In [18]:
# Especifica la ruta del archivo CSV que deseas leer
file_path = 'Data/Datos_csv/df_train_filtrado.csv'

# Lee el archivo CSV y crea un DataFrame
df_train_filtrado = pd.read_csv(file_path)

# Ahora puedes trabajar con el DataFrame 'reviews'
print(df_train_filtrado)

          user_id  item_id  recommend
0       diego9031   223710       True
1       diego9031   299360       True
2       diego9031   322330       True
3       diego9031     1250       True
4       diego9031    12100       True
...           ...      ...        ...
5689  ShadowYT100   354500       True
5690  ShadowYT100    10180       True
5691  ShadowYT100   301520       True
5692  ShadowYT100      440       True
5693  ShadowYT100   265630       True

[5694 rows x 3 columns]


# Importamos las librerias necesarias para entrenar y evaluar el modelo

In [19]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic

# Definimos el conjunto de prueba y entrenamiento y entrenamos el modelo

In [20]:
# Configura el lector y la escala de calificación (ajusta esto según tus datos)
reader = Reader(rating_scale=(0, 1))

# Carga los datos
data = Dataset.load_from_df(df_train_filtrado[['user_id', 'item_id', 'recommend']], reader)

# Divide los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

# Crea el modelo KNN (K-Nearest Neighbors) para encontrar usuarios similares
sim_options = {'name': 'pearson', 'user_based': True}
algo = KNNBasic(sim_options=sim_options)

# Entrena el modelo en el conjunto de entrenamiento
algo.fit(trainset)


Computing the pearson similarity matrix...
Done computing similarity matrix.


# Guardamos el modelo entrenado en un archivo binario para su posterior uso

In [21]:
import joblib

# Guardar el modelo
joblib.dump(algo, 'modelo_surprise.pkl')

['modelo_surprise.pkl']

# Cargamos el modelo

In [22]:
import joblib
# Cargar el modelo
algo = joblib.load('modelo_surprise.pkl')

# Obtenimos recomendaciones para un usuario dado.

In [23]:
# Supongamos que tenemos un usuario de interés con user_id = 'tu_usuario'
tu_usuario = 'ShadowYT100'

# Encuentra los usuarios más similares al usuario de interés
similar_users = algo.get_neighbors(algo.trainset.to_inner_uid(tu_usuario), k=10)

# Filtra las reseñas de los usuarios similares
reviews_similares = reviews[reviews['user_id'].isin([algo.trainset.to_raw_uid(uid) for uid in similar_users])]

# Filtra las reseñas de juegos que han sido recomendados (recommend=True)
juegos_recomendados = reviews_similares[reviews_similares['recommend']]

# Obtén la lista de juegos recomendados por usuarios similares
juegos_recomendados_lista = juegos_recomendados['item_id'].unique()

# Imprime la lista de juegos recomendados
print("Juegos recomendados por usuarios similares:")
print(juegos_recomendados_lista)

Juegos recomendados por usuarios similares:
[ 17390 304930 346900 246620 290140 105600 219640 109400    440 359550
   8930  72850 244850 377160  35450 224260   4920    550   1250   8190
 230410  50620 235540 238320 252490   8870  49520 310380    730 412470
   4000 206420  24800 219740  17480 206500  99900  24980    570 200710
    240    420    380    620    400 207610 244770 306130 391540 417860
 324810 247000  65800  55230 241540 206210  24240]


In [24]:
similar_users

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [25]:
reviews_similares.head(3)

,user_id,posted,item_id,recommend,review,sentiment_score,sentiment_analysis
10324,76561198064394417,2015-12-04,17390,True,I made a human10/10,0.0000,1
10325,76561198064394417,2015-12-04,304930,True,Zombies100/10,0.0000,1
10326,76561198064394417,2015-11-05,346900,True,"Money, fake money ftw.9/10",-0.4767,0


# Evaluamos el rendimiento del modelo

In [26]:
# Realiza predicciones en el conjunto de prueba
predictions = algo.test(testset)

# Calcula la RMSE para evaluar el rendimiento del modelo
rmse = accuracy.rmse(predictions)

# Imprime la RMSE
print(f'RMSE: {rmse}')

RMSE: 0.3176
RMSE: 0.31760005558055765


# Obtenemos un listado con los 5 juego recomendados para el usuario dado apartir de las recomendaciones hechas por el modelo.

In [27]:
# Inicializa una lista vacía para almacenar los juegos encontrados
juegos_encontrados = []

# Itera a través de la lista de juegos recomendados
for juego_id in juegos_recomendados_lista:
    # Busca el juego en el DataFrame games
    juego = games[games['item_id'] == juego_id]
    
    # Si se encuentra el juego, agrégalo a la lista de juegos encontrados
    if not juego.empty:
        juegos_encontrados.append(juego[['item_id', 'app_name']])  # Agrega solo las columnas "item_id" y "app_name" del juego encontrado

    # Si ya hemos encontrado 5 juegos, detén el bucle
    if len(juegos_encontrados) == 5:
        break

# Convierte la lista de juegos encontrados en un DataFrame
juegos_encontrados_df = pd.concat(juegos_encontrados, ignore_index=True)

# Imprime el DataFrame de juegos encontrados con las columnas "item_id" y "app_name"
print(juegos_encontrados_df[['item_id', 'app_name']])
juegos_encontrados_df

   item_id              app_name
0    17390                SPORE™
1   304930              Unturned
2   346900  AdVenture Capitalist
3   246620   Plague Inc: Evolved
4   290140          Echo of Soul


,item_id,app_name
0,17390,SPORE™
1,304930,Unturned
2,346900,AdVenture Capitalist
3,246620,Plague Inc: Evolved
4,290140,Echo of Soul


# Con todo lo anterior creamos una funcion que hace posible consumir el modelo de recomendacion.

In [28]:
def recomendacion_usuario1(id_de_usuario: str):

    # Encuentra los usuarios más similares al usuario de interés
    similar_users = algo.get_neighbors(algo.trainset.to_inner_uid(id_de_usuario), k=10)

    # Filtra las reseñas de los usuarios similares
    reviews_similares = reviews[reviews['user_id'].isin([algo.trainset.to_raw_uid(uid) for uid in similar_users])]

    # Filtra las reseñas de juegos que han sido recomendados (recommend=True)
    juegos_recomendados = reviews_similares[reviews_similares['recommend']]

    # Obtén la lista de juegos recomendados por usuarios similares
    juegos_recomendados_lista = juegos_recomendados['item_id'].unique()

    # Inicializa una lista vacía para almacenar los juegos encontrados
    juegos_encontrados = []

    # Itera a través de la lista de juegos recomendados
    for juego_id in juegos_recomendados_lista:
        # Busca el juego en el DataFrame games
        juego = games[games['item_id'] == juego_id]
        
        # Si se encuentra el juego, agrégalo a la lista de juegos encontrados
        if not juego.empty:
            juegos_encontrados.append(juego[['item_id', 'app_name']])  # Agrega solo las columnas "item_id" y "app_name" del juego encontrado

        # Si ya hemos encontrado 5 juegos, detén el bucle
        if len(juegos_encontrados) == 5:
            break

    # Convierte la lista de juegos encontrados en un DataFrame
    juegos_encontrados_df = pd.concat(juegos_encontrados, ignore_index=True)

    # Imprime el DataFrame de juegos encontrados con las columnas "item_id" y "app_name"
    juegos_encontrados = juegos_encontrados_df[['item_id', 'app_name']].to_dict(orient='records')
    return juegos_encontrados

In [29]:
recomendacion_usuario1('ShadowYT100')

[{'item_id': 17390, 'app_name': 'SPORE™'},
 {'item_id': 304930, 'app_name': 'Unturned'},
 {'item_id': 346900, 'app_name': 'AdVenture Capitalist'},
 {'item_id': 246620, 'app_name': 'Plague Inc: Evolved'},
 {'item_id': 290140, 'app_name': 'Echo of Soul'}]